In [1]:
# Enhanced Customer Data Processing and Segmentation with SQL Functions

import sqlite3

def create_tables(cursor):
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Customers (
            CustomerNumber INTEGER PRIMARY KEY,
            Name TEXT,
            Age INTEGER,
            ZipCode TEXT
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS CustomerInfo (
            CustomerNumber INTEGER PRIMARY KEY,
            Income INTEGER,
            CreditScore INTEGER
        )
    ''')

def insert_data(cursor, customers, customer_info):
    cursor.executemany('''
        INSERT OR REPLACE INTO Customers (CustomerNumber, Name, Age, ZipCode)
        VALUES (?, ?, ?, ?)
    ''', customers)

    cursor.executemany('''
        INSERT OR REPLACE INTO CustomerInfo (CustomerNumber, Income, CreditScore)
        VALUES (?, ?, ?)
    ''', customer_info)

def create_summary_tables(cursor):
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS CustomerSummary AS
        SELECT
            Customers.CustomerNumber,
            Customers.Name,
            AVG(Customers.Age) AS AverageAge,
            Customers.ZipCode,
            AVG(CustomerInfo.Income) AS AverageIncome,
            AVG(CustomerInfo.CreditScore) AS AverageCreditScore
        FROM
            Customers
        INNER JOIN
            CustomerInfo
        ON
            Customers.CustomerNumber = CustomerInfo.CustomerNumber
        GROUP BY
            Customers.CustomerNumber, Customers.Name, Customers.ZipCode
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS SummaryStatistics AS
        SELECT
            'Customers' AS TableName,
            'Age' AS ColumnName,
            COUNT(Age) AS RowCount,
            AVG(Age) AS Average,
            SUM(Age) AS Sum,
            MIN(Age) AS Min,
            MAX(Age) AS Max
        FROM
            Customers
        UNION ALL
        SELECT
            'CustomerInfo' AS TableName,
            'Income' AS ColumnName,
            COUNT(Income) AS RowCount,
            AVG(Income) AS Average,
            SUM(Income) AS Sum,
            MIN(Income) AS Min,
            MAX(Income) AS Max
        FROM
            CustomerInfo
        UNION ALL
        SELECT
            'CustomerInfo' AS TableName,
            'CreditScore' AS ColumnName,
            COUNT(CreditScore) AS RowCount,
            AVG(CreditScore) AS Average,
            SUM(CreditScore) AS Sum,
            MIN(CreditScore) AS Min,
            MAX(CreditScore) AS Max
        FROM
            CustomerInfo
    ''')

def main():
    customers = [
        (1, 'Alice', 30, '12345'),
        (1, 'Alice', 31, '12345'),
        (2, 'Bob', 25, '23456'),
        (2, 'Bob', 26, '23456'),
        (3, 'Charlie', 35, '34567'),
        (3, 'Charlie', 36, '34567'),
        (4, 'David', 28, '45678'),
        (4, 'David', 29, '45678'),
        (5, 'Eve', 32, '56789'),
        (5, 'Eve', 33, '56789'),
        (6, 'Frank', 45, '67890'),
        (6, 'Frank', 46, '67890'),
        (7, 'Grace', 27, '78901'),
        (7, 'Grace', 28, '78901'),
        (8, 'Hannah', 29, '89012'),
        (8, 'Hannah', 30, '89012'),
        (9, 'Isaac', 40, '90123'),
        (9, 'Isaac', 41, '90123'),
        (10, 'Jack', 22, '01234'),
        (10, 'Jack', 23, '01234')
    ]

    customer_info = [
        (1, 55000, 700),
        (1, 56000, 705),
        (2, 62000, 680),
        (2, 63000, 685),
        (3, 48000, 710),
        (3, 49000, 715),
        (4, 75000, 690),
        (4, 76000, 695),
        (5, 50000, 720),
        (5, 51000, 725),
        (6, 82000, 710),
        (6, 83000, 715),
        (7, 53000, 680),
        (7, 54000, 685),
        (8, 61000, 700),
        (8, 62000, 705),
        (9, 47000, 690),
        (9, 48000, 695),
        (10, 58000, 730),
        (10, 59000, 735)
    ]

    with sqlite3.connect('customer_data.db') as conn:
        cursor = conn.cursor()

        try:
            create_tables(cursor)
            insert_data(cursor, customers, customer_info)
            conn.commit()

            create_summary_tables(cursor)
            conn.commit()

            cursor.execute('SELECT * FROM CustomerSummary')
            summary_results = cursor.fetchall()

            print("Customer Summary Data:")
            for row in summary_results:
                print(row)

            cursor.execute('SELECT * FROM SummaryStatistics')
            statistics_results = cursor.fetchall()

            print("\nSummary Statistics Data:")
            print(f"{'TableName':<15} {'ColumnName':<15} {'RowCount':<10} {'Average':<10} {'Sum':<10} {'Min':<10} {'Max':<10}")
            for row in statistics_results:
                print(f"{row[0]:<15} {row[1]:<15} {row[2]:<10} {row[3]:<10} {row[4]:<10} {row[5]:<10} {row[6]:<10}")

            cursor.execute('''
                SELECT Customers.Name, CustomerInfo.Income, CustomerInfo.CreditScore
                FROM Customers
                JOIN CustomerInfo ON Customers.CustomerNumber = CustomerInfo.CustomerNumber
                WHERE CustomerInfo.Income > (SELECT AVG(Income) FROM CustomerInfo)
                AND CustomerInfo.CreditScore > (SELECT AVG(CreditScore) FROM CustomerInfo)
            ''')
            high_income_customers = cursor.fetchall()
            print("\nHigh Income Customers with Good Credit Scores:")
            for row in high_income_customers:
                print(row)

        except sqlite3.Error as e:
            print(f"An error occurred: {e}")

if __name__ == '__main__':
    main()


Customer Summary Data:
(1, 'Alice', 31.0, '12345', 56000.0, 705.0)
(2, 'Bob', 26.0, '23456', 63000.0, 685.0)
(3, 'Charlie', 36.0, '34567', 49000.0, 715.0)
(4, 'David', 29.0, '45678', 76000.0, 695.0)
(5, 'Eve', 33.0, '56789', 51000.0, 725.0)
(6, 'Frank', 46.0, '67890', 83000.0, 715.0)
(7, 'Grace', 28.0, '78901', 54000.0, 685.0)
(8, 'Hannah', 30.0, '89012', 62000.0, 705.0)
(9, 'Isaac', 41.0, '90123', 48000.0, 695.0)
(10, 'Jack', 23.0, '01234', 59000.0, 735.0)

Summary Statistics Data:
TableName       ColumnName      RowCount   Average    Sum        Min        Max       
Customers       Age             10         32.3       323        23         46        
CustomerInfo    Income          10         60100.0    601000     48000      83000     
CustomerInfo    CreditScore     10         706.0      7060       685        735       

High Income Customers with Good Credit Scores:
('Frank', 83000, 715)
